# 🚀 SVM+HOG vs CNN Image Classification Comparison

This notebook provides a comprehensive comparison between two different image classification approaches:
- **SVM + HOG** (Traditional Computer Vision)
- **CNN** (Deep Learning)

## 📊 Features
- Comprehensive accuracy, training time, and inference time analysis
- Automatic GPU detection and utilization
- Detailed visualizations and confusion matrices
- Separate downloads for models, plots, and results
- Configurable hyperparameters for both approaches

## 🎯 Dataset
- **Classes**: 3 (normal, cheating, looking_around)
- **Images**: 150 total (50 per class)
- **Format**: PNG images (128x128)
- **Type**: Synthetic dataset with distinct visual patterns


## 🔧 Environment Setup and Dependencies

In [ ]:
# Install required packages
!pip install opencv-python-headless scikit-image tqdm

# Import all required libraries
import os
import sys
import numpy as np
import cv2
import time
import json
import zipfile
import warnings
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

# Machine Learning imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from skimage.feature import hog

# Suppress warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

print("✅ All dependencies installed and imported successfully!")

## 🎮 GPU Detection and Configuration

In [ ]:
# GPU Detection and Configuration
print("🔍 Checking GPU availability...")
print(f"TensorFlow version: {tf.__version__}")

# Check GPU availability
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Enable memory growth for GPU
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"🚀 GPU detected and configured! Available GPUs: {len(gpus)}")
        for i, gpu in enumerate(gpus):
            print(f"   GPU {i}: {gpu.name}")
    except RuntimeError as e:
        print(f"⚠️ GPU configuration error: {e}")
else:
    print("💻 No GPU detected. Running on CPU.")

# Check if GPU is being used
device_name = tf.test.gpu_device_name()
if device_name:
    print(f"🎯 Using GPU: {device_name}")
else:
    print("🖥️ Using CPU for computations")

# Set mixed precision for better GPU performance
if gpus:
    try:
        policy = tf.keras.mixed_precision.Policy('mixed_float16')
        tf.keras.mixed_precision.set_global_policy(policy)
        print("⚡ Mixed precision enabled for better GPU performance")
    except:
        print("⚠️ Mixed precision not available, using default precision")

## ⚙️ Configuration Settings

In [ ]:
# Configuration Settings
# Dataset Configuration
DATASET_CONFIG = {
    'image_size': (128, 128),
    'test_size': 0.2,
    'random_state': 42,
    'supported_formats': ['.png'],
}

# SVM + HOG Configuration
SVM_HOG_CONFIG = {
    'hog_params': {
        'orientations': 9,
        'pixels_per_cell': (8, 8),
        'cells_per_block': (2, 2),
        'block_norm': 'L2-Hys',
        'visualize': False,
    },
    'svm_params': {
        'C': 1.0,
        'kernel': 'rbf',
        'gamma': 'scale',
        'random_state': 42,
        'probability': True,
    },
    'scaler': 'StandardScaler'
}

# CNN Configuration
CNN_CONFIG = {
    'architecture': {
        'input_shape': (128, 128, 3),
        'conv_layers': [
            {'filters': 32, 'kernel_size': (3, 3), 'activation': 'relu'},
            {'filters': 64, 'kernel_size': (3, 3), 'activation': 'relu'},
            {'filters': 128, 'kernel_size': (3, 3), 'activation': 'relu'},
        ],
        'dense_layers': [
            {'units': 128, 'activation': 'relu', 'dropout': 0.5},
            {'units': 64, 'activation': 'relu', 'dropout': 0.3},
        ],
        'output_activation': 'softmax'
    },
    'compilation': {
        'optimizer': 'adam',
        'loss': 'categorical_crossentropy',
        'metrics': ['accuracy'],
        'learning_rate': 0.001
    },
    'training': {
        'epochs': 50,
        'batch_size': 32,
        'validation_split': 0.2,
        'early_stopping': {
            'monitor': 'val_accuracy',
            'patience': 10,
            'restore_best_weights': True
        }
    },
    'augmentation': {
        'rotation_range': 20,
        'width_shift_range': 0.2,
        'height_shift_range': 0.2,
        'shear_range': 0.2,
        'zoom_range': 0.2,
        'horizontal_flip': True,
        'fill_mode': 'nearest'
    }
}

# Evaluation Configuration
EVALUATION_CONFIG = {
    'metrics': ['accuracy', 'precision', 'recall', 'f1-score'],
    'figure_size': (12, 8),
    'save_plots': True,
}

# Timing Configuration
TIMING_CONFIG = {
    'inference_samples': 100,
    'timing_runs': 5,
}

print("✅ Configuration loaded successfully!")
print(f"📊 Image size: {DATASET_CONFIG['image_size']}")
print(f"🧠 CNN epochs: {CNN_CONFIG['training']['epochs']}")
print(f"⚙️ SVM kernel: {SVM_HOG_CONFIG['svm_params']['kernel']}")

## 📁 Dataset Creation and Loading

In [ ]:
# Dataset Creation Function
def create_synthetic_dataset():
    """Create a synthetic dataset with distinct visual patterns for each class"""
    
    # Create dataset directory structure
    dataset_dir = 'dataset'
    classes = ['normal', 'cheating', 'looking_around']
    
    # Create directories
    for class_name in classes:
        class_dir = os.path.join(dataset_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)
    
    # Generate sample images for each class
    image_size = DATASET_CONFIG['image_size']
    num_images_per_class = 50
    
    for i, class_name in enumerate(classes):
        print(f"Creating {num_images_per_class} sample images for class '{class_name}'...")
        
        for j in tqdm(range(num_images_per_class), desc=f"{class_name}"):
            # Create a synthetic image with different patterns for each class
            img = np.zeros((*image_size, 3), dtype=np.uint8)
            
            if class_name == 'normal':
                # Normal: Blue-ish with some noise
                img[:, :, 2] = 150 + np.random.randint(0, 50, image_size)  # Blue channel
                img[:, :, 1] = 50 + np.random.randint(0, 30, image_size)   # Green channel
                img[:, :, 0] = 30 + np.random.randint(0, 20, image_size)   # Red channel
                
            elif class_name == 'cheating':
                # Cheating: Red-ish with specific patterns
                img[:, :, 0] = 150 + np.random.randint(0, 50, image_size)  # Red channel
                img[:, :, 1] = 30 + np.random.randint(0, 20, image_size)   # Green channel
                img[:, :, 2] = 30 + np.random.randint(0, 20, image_size)   # Blue channel
                
                # Add some diagonal patterns
                for k in range(0, image_size[0], 10):
                    img[k:k+2, :, :] = 255
                
            elif class_name == 'looking_around':
                # Looking around: Green-ish with circular patterns
                img[:, :, 1] = 150 + np.random.randint(0, 50, image_size)  # Green channel
                img[:, :, 0] = 30 + np.random.randint(0, 20, image_size)   # Red channel
                img[:, :, 2] = 30 + np.random.randint(0, 20, image_size)   # Blue channel
                
                # Add some circular patterns
                center = (image_size[0] // 2, image_size[1] // 2)
                for radius in range(10, 50, 10):
                    for angle in range(0, 360, 10):
                        x = int(center[0] + radius * np.cos(np.radians(angle)))
                        y = int(center[1] + radius * np.sin(np.radians(angle)))
                        if 0 <= x < image_size[0] and 0 <= y < image_size[1]:
                            img[x-1:x+2, y-1:y+2, :] = 255
            
            # Convert to PIL Image and save as PNG
            pil_img = Image.fromarray(img)
            filename = f"{class_name}_{j+1:03d}.png"
            filepath = os.path.join(dataset_dir, class_name, filename)
            pil_img.save(filepath)
    
    print(f"✅ Synthetic dataset created successfully!")
    print(f"📁 Dataset location: {dataset_dir}")
    print(f"📊 Classes: {classes}")
    print(f"🖼️ Images per class: {num_images_per_class}")
    print(f"📏 Image size: {image_size}")
    
    return dataset_dir, classes

# Create the dataset
dataset_dir, class_names = create_synthetic_dataset()

In [ ]:
# Visualize Sample Images from Dataset
def visualize_sample_images(dataset_dir, class_names, samples_per_class=3):
    """Visualize sample images from each class"""
    
    fig, axes = plt.subplots(len(class_names), samples_per_class, 
                            figsize=(15, 5 * len(class_names)))
    
    for i, class_name in enumerate(class_names):
        class_dir = os.path.join(dataset_dir, class_name)
        image_files = [f for f in os.listdir(class_dir) if f.endswith('.png')][:samples_per_class]
        
        for j, img_file in enumerate(image_files):
            img_path = os.path.join(class_dir, img_file)
            img = Image.open(img_path)
            
            axes[i, j].imshow(img)
            axes[i, j].set_title(f"{class_name} - Sample {j+1}")
            axes[i, j].axis('off')
    
    plt.suptitle("Sample Images from Each Class", fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Visualize the dataset
print("📸 Visualizing sample images from the dataset...")
visualize_sample_images(dataset_dir, class_names)

## 📊 Data Loading and Preprocessing

In [ ]:
# Data Loading and Preprocessing Class
class ImageDataLoader:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.image_size = DATASET_CONFIG['image_size']
        self.test_size = DATASET_CONFIG['test_size']
        self.random_state = DATASET_CONFIG['random_state']
        self.supported_formats = DATASET_CONFIG['supported_formats']
        
        self.label_encoder = LabelEncoder()
        self.class_names = []
        
    def load_dataset(self):
        """Load images from subdirectories organized by class"""
        print("📂 Loading dataset...")
        
        if not os.path.exists(self.data_dir):
            raise FileNotFoundError(f"Dataset directory '{self.data_dir}' not found!")
        
        images = []
        labels = []
        
        # Get class directories
        class_dirs = [d for d in os.listdir(self.data_dir) 
                     if os.path.isdir(os.path.join(self.data_dir, d))]
        
        if not class_dirs:
            raise ValueError(f"No class directories found in '{self.data_dir}'")
        
        self.class_names = sorted(class_dirs)
        print(f"Found {len(self.class_names)} classes: {self.class_names}")
        
        # Load images from each class
        for class_name in tqdm(self.class_names, desc="Loading classes"):
            class_path = os.path.join(self.data_dir, class_name)
            
            # Get all supported image files
            image_files = [f for f in os.listdir(class_path) 
                          if any(f.lower().endswith(ext) for ext in self.supported_formats)]
            
            if not image_files:
                print(f"Warning: No supported images found in '{class_path}'")
                continue
            
            print(f"  - {class_name}: {len(image_files)} images")
            
            for img_file in image_files:
                img_path = os.path.join(class_path, img_file)
                try:
                    # Load and preprocess image
                    image = self._load_and_preprocess_image(img_path)
                    if image is not None:
                        images.append(image)
                        labels.append(class_name)
                except Exception as e:
                    print(f"Error loading {img_path}: {e}")
                    continue
        
        if not images:
            raise ValueError("No images were successfully loaded!")
        
        # Convert to numpy arrays
        X = np.array(images)
        y = np.array(labels)
        
        # Encode labels
        y_encoded = self.label_encoder.fit_transform(y)
        
        print(f"✅ Dataset loaded: {len(X)} images, {len(self.class_names)} classes")
        print(f"📊 Image shape: {X.shape}")
        
        return X, y_encoded, self.class_names
    
    def _load_and_preprocess_image(self, img_path):
        """Load and preprocess a single image"""
        try:
            # Load image using PIL
            img = Image.open(img_path)
            
            # Convert to RGB if needed
            if img.mode != 'RGB':
                img = img.convert('RGB')
            
            # Resize image
            img = img.resize(self.image_size, Image.Resampling.LANCZOS)
            
            # Convert to numpy array
            img_array = np.array(img)
            
            # Normalize pixel values to [0, 1]
            img_array = img_array.astype(np.float32) / 255.0
            
            return img_array
            
        except Exception as e:
            print(f"Error processing {img_path}: {e}")
            return None
    
    def prepare_data_for_svm(self, X, y):
        """Prepare data for SVM (flatten images)"""
        # Flatten images for SVM
        X_flat = X.reshape(X.shape[0], -1)
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X_flat, y, test_size=self.test_size, 
            random_state=self.random_state, stratify=y
        )
        
        return X_train, X_test, y_train, y_test
    
    def prepare_data_for_cnn(self, X, y):
        """Prepare data for CNN (keep image structure)"""
        # Convert labels to categorical
        y_categorical = to_categorical(y, num_classes=len(self.class_names))
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_categorical, test_size=self.test_size, 
            random_state=self.random_state, stratify=y
        )
        
        return X_train, X_test, y_train, y_test
    
    def create_data_generator(self, X_train, y_train, batch_size=32):
        """Create data generator with augmentation for CNN"""
        datagen = ImageDataGenerator(**CNN_CONFIG['augmentation'])
        
        # Fit the generator to training data
        datagen.fit(X_train)
        
        # Create generator
        generator = datagen.flow(X_train, y_train, batch_size=batch_size)
        
        return generator
    
    def get_class_distribution(self, y):
        """Get class distribution for analysis"""
        unique, counts = np.unique(y, return_counts=True)
        distribution = dict(zip(unique, counts))
        
        print("\n📊 Class Distribution:")
        for i, class_name in enumerate(self.class_names):
            count = distribution.get(i, 0)
            print(f"  {class_name}: {count} images")
        
        return distribution

# Load and prepare data
print("🔄 Loading and preparing data...")
data_loader = ImageDataLoader(data_dir=dataset_dir)

# Load raw data
X, y, class_names = data_loader.load_dataset()

# Show class distribution
data_loader.get_class_distribution(y)

# Prepare data for both models
X_train_svm, X_test_svm, y_train_svm, y_test_svm = data_loader.prepare_data_for_svm(X, y)
X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = data_loader.prepare_data_for_cnn(X, y)

print(f"\n✅ Data prepared successfully!")
print(f"   Training samples: {len(X_train_svm)}")
print(f"   Test samples: {len(X_test_svm)}")
print(f"   Classes: {len(class_names)}")
print(f"   Class names: {class_names}")

## 🔧 SVM + HOG Model Implementation

In [ ]:
# SVM + HOG Model Class
class SVMHOGModel:
    def __init__(self, config=None):
        self.config = config or SVM_HOG_CONFIG
        self.hog_params = self.config['hog_params']
        self.svm_params = self.config['svm_params']
        
        # Initialize pipeline
        self.pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('svm', SVC(**self.svm_params))
        ])
        
        self.is_trained = False
        self.training_time = 0
        
    def extract_hog_features(self, images):
        """Extract HOG features from images"""
        print("🔍 Extracting HOG features...")
        
        features = []
        for img in tqdm(images, desc="HOG extraction"):
            try:
                # Convert to grayscale if needed for HOG
                if len(img.shape) == 3:
                    # Convert RGB to grayscale
                    gray = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
                else:
                    gray = img
                
                # Ensure image is valid for HOG
                if gray.shape[0] < 32 or gray.shape[1] < 32:
                    print(f"Warning: Image too small for HOG: {gray.shape}")
                    features.append(np.zeros(1296))  # Default HOG feature size
                    continue
                
                # Extract HOG features
                hog_features = hog(
                    gray,
                    orientations=self.hog_params['orientations'],
                    pixels_per_cell=self.hog_params['pixels_per_cell'],
                    cells_per_block=self.hog_params['cells_per_block'],
                    block_norm=self.hog_params['block_norm'],
                    visualize=self.hog_params['visualize']
                )
                
                features.append(hog_features)
                
            except Exception as e:
                print(f"Error extracting HOG features: {e}")
                # Use zero features as fallback
                features.append(np.zeros(1296))  # Default HOG feature size
        
        return np.array(features)
    
    def train(self, X_train, y_train):
        """Train the SVM + HOG model"""
        print("🔧 Training SVM + HOG model...")
        
        # Extract HOG features
        X_train_hog = self.extract_hog_features(X_train)
        
        # Train the pipeline
        start_time = time.time()
        self.pipeline.fit(X_train_hog, y_train)
        self.training_time = time.time() - start_time
        
        self.is_trained = True
        print(f"✅ Training completed in {self.training_time:.2f} seconds")
        
        return self
    
    def predict(self, X_test):
        """Make predictions"""
        if not self.is_trained:
            raise ValueError("Model must be trained before making predictions")
        
        # Extract HOG features
        X_test_hog = self.extract_hog_features(X_test)
        
        # Make predictions
        predictions = self.pipeline.predict(X_test_hog)
        
        return predictions
    
    def predict_proba(self, X_test):
        """Get prediction probabilities"""
        if not self.is_trained:
            raise ValueError("Model must be trained before making predictions")
        
        # Extract HOG features
        X_test_hog = self.extract_hog_features(X_test)
        
        # Get probabilities
        probabilities = self.pipeline.predict_proba(X_test_hog)
        
        return probabilities
    
    def evaluate(self, X_test, y_test, class_names):
        """Evaluate model performance"""
        print("📊 Evaluating SVM + HOG model...")
        
        # Make predictions
        start_time = time.time()
        predictions = self.predict(X_test)
        inference_time = (time.time() - start_time) / len(X_test)
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, predictions)
        
        # Generate classification report
        report = classification_report(
            y_test, predictions, 
            target_names=class_names, 
            output_dict=True
        )
        
        results = {
            'accuracy': accuracy,
            'classification_report': report,
            'training_time': self.training_time,
            'inference_time_per_sample': inference_time,
            'predictions': predictions,
            'model_name': 'SVM + HOG'
        }
        
        print(f"📈 SVM + HOG Results:")
        print(f"  Accuracy: {accuracy:.4f}")
        print(f"  Training Time: {self.training_time:.2f} seconds")
        print(f"  Inference Time: {inference_time:.6f} seconds per sample")
        
        return results
    
    def measure_inference_time(self, X_sample, num_runs=5):
        """Measure detailed inference time"""
        if not self.is_trained:
            raise ValueError("Model must be trained before measuring inference time")
        
        times = []
        for _ in range(num_runs):
            start_time = time.time()
            _ = self.predict(X_sample)
            end_time = time.time()
            times.append((end_time - start_time) / len(X_sample))
        
        return {
            'mean_time': np.mean(times),
            'std_time': np.std(times),
            'min_time': np.min(times),
            'max_time': np.max(times)
        }
    
    def get_model_info(self):
        """Get model configuration information"""
        return {
            'model_type': 'SVM + HOG',
            'hog_params': self.hog_params,
            'svm_params': self.svm_params,
            'is_trained': self.is_trained,
            'training_time': self.training_time
        }

print("✅ SVM + HOG model class defined!")

## 🧠 CNN Model Implementation

In [ ]:
# CNN Model Class
class CNNModel:
    def __init__(self, num_classes, config=None):
        self.config = config or CNN_CONFIG
        self.num_classes = num_classes
        self.model = None
        self.history = None
        self.is_trained = False
        self.training_time = 0
        
        # Build model
        self._build_model()
    
    def _build_model(self):
        """Build CNN architecture"""
        print("🏗️ Building CNN model...")
        
        self.model = Sequential()
        
        # Input layer
        input_shape = self.config['architecture']['input_shape']
        
        # Convolutional layers
        for i, conv_config in enumerate(self.config['architecture']['conv_layers']):
            if i == 0:
                # First layer needs input shape
                self.model.add(Conv2D(
                    filters=conv_config['filters'],
                    kernel_size=conv_config['kernel_size'],
                    activation=conv_config['activation'],
                    input_shape=input_shape
                ))
            else:
                self.model.add(Conv2D(
                    filters=conv_config['filters'],
                    kernel_size=conv_config['kernel_size'],
                    activation=conv_config['activation']
                ))
            
            # Add pooling after each conv layer
            self.model.add(MaxPooling2D(pool_size=(2, 2)))
        
        # Flatten before dense layers
        self.model.add(Flatten())
        
        # Dense layers
        for dense_config in self.config['architecture']['dense_layers']:
            self.model.add(Dense(
                units=dense_config['units'],
                activation=dense_config['activation']
            ))
            
            # Add dropout if specified
            if 'dropout' in dense_config:
                self.model.add(Dropout(dense_config['dropout']))
        
        # Output layer
        self.model.add(Dense(
            units=self.num_classes,
            activation=self.config['architecture']['output_activation']
        ))
        
        # Compile model
        optimizer = Adam(learning_rate=self.config['compilation']['learning_rate'])
        
        self.model.compile(
            optimizer=optimizer,
            loss=self.config['compilation']['loss'],
            metrics=self.config['compilation']['metrics']
        )
        
        print("✅ CNN model built successfully!")
        print(f"📊 Total parameters: {self.model.count_params():,}")
        
        # Display model summary
        self.model.summary()
    
    def train(self, X_train, y_train, X_val=None, y_val=None, data_generator=None):
        """Train the CNN model"""
        print("🧠 Training CNN model...")
        
        # Setup callbacks
        callbacks = []
        
        # Early stopping
        early_stopping_config = self.config['training']['early_stopping']
        early_stopping = EarlyStopping(
            monitor=early_stopping_config['monitor'],
            patience=early_stopping_config['patience'],
            restore_best_weights=early_stopping_config['restore_best_weights']
        )
        callbacks.append(early_stopping)
        
        # Training parameters
        epochs = self.config['training']['epochs']
        batch_size = self.config['training']['batch_size']
        validation_split = self.config['training']['validation_split']
        
        start_time = time.time()
        
        if data_generator is not None:
            # Train with data augmentation
            print("🔄 Training with data augmentation...")
            
            # Calculate steps
            steps_per_epoch = len(X_train) // batch_size
            
            # Validation data
            if X_val is not None and y_val is not None:
                validation_data = (X_val, y_val)
            else:
                validation_data = None
            
            self.history = self.model.fit(
                data_generator,
                epochs=epochs,
                steps_per_epoch=steps_per_epoch,
                validation_data=validation_data,
                callbacks=callbacks,
                verbose=1
            )
        else:
            # Train without data augmentation
            print("📚 Training without data augmentation...")
            
            self.history = self.model.fit(
                X_train, y_train,
                epochs=epochs,
                batch_size=batch_size,
                validation_split=validation_split,
                callbacks=callbacks,
                verbose=1
            )
        
        self.training_time = time.time() - start_time
        self.is_trained = True
        
        print(f"✅ Training completed in {self.training_time:.2f} seconds")
        
        return self
    
    def predict(self, X_test):
        """Make predictions"""
        if not self.is_trained:
            raise ValueError("Model must be trained before making predictions")
        
        # Get predictions
        predictions = self.model.predict(X_test)
        
        # Convert probabilities to class predictions
        predicted_classes = np.argmax(predictions, axis=1)
        
        return predicted_classes
    
    def predict_proba(self, X_test):
        """Get prediction probabilities"""
        if not self.is_trained:
            raise ValueError("Model must be trained before making predictions")
        
        probabilities = self.model.predict(X_test)
        
        return probabilities
    
    def evaluate(self, X_test, y_test, class_names):
        """Evaluate model performance"""
        print("📊 Evaluating CNN model...")
        
        # Make predictions
        start_time = time.time()
        predictions = self.predict(X_test)
        inference_time = (time.time() - start_time) / len(X_test)
        
        # Convert categorical y_test to class indices if needed
        if len(y_test.shape) > 1:
            y_test_classes = np.argmax(y_test, axis=1)
        else:
            y_test_classes = y_test
        
        # Calculate metrics
        accuracy = accuracy_score(y_test_classes, predictions)
        
        # Generate classification report
        report = classification_report(
            y_test_classes, predictions, 
            target_names=class_names, 
            output_dict=True
        )
        
        results = {
            'accuracy': accuracy,
            'classification_report': report,
            'training_time': self.training_time,
            'inference_time_per_sample': inference_time,
            'predictions': predictions,
            'model_name': 'CNN',
            'history': self.history.history if self.history else None
        }
        
        print(f"📈 CNN Results:")
        print(f"  Accuracy: {accuracy:.4f}")
        print(f"  Training Time: {self.training_time:.2f} seconds")
        print(f"  Inference Time: {inference_time:.6f} seconds per sample")
        
        return results
    
    def measure_inference_time(self, X_sample, num_runs=5):
        """Measure detailed inference time"""
        if not self.is_trained:
            raise ValueError("Model must be trained before measuring inference time")
        
        times = []
        for _ in range(num_runs):
            start_time = time.time()
            _ = self.predict(X_sample)
            end_time = time.time()
            times.append((end_time - start_time) / len(X_sample))
        
        return {
            'mean_time': np.mean(times),
            'std_time': np.std(times),
            'min_time': np.min(times),
            'max_time': np.max(times)
        }
    
    def get_model_info(self):
        """Get model configuration information"""
        return {
            'model_type': 'CNN',
            'architecture': self.config['architecture'],
            'compilation': self.config['compilation'],
            'training': self.config['training'],
            'augmentation': self.config['augmentation'],
            'is_trained': self.is_trained,
            'training_time': self.training_time,
            'num_parameters': self.model.count_params() if self.model else 0
        }

print("✅ CNN model class defined!")

## 🚀 Model Training

In [ ]:
# Train SVM + HOG Model
print("🔧 Training SVM + HOG Model...")
print("="*50)

svm_model = SVMHOGModel(config=SVM_HOG_CONFIG)
svm_model.train(X_train_svm, y_train_svm)

print("\n✅ SVM + HOG model training completed!")

In [ ]:
# Train CNN Model
print("🧠 Training CNN Model...")
print("="*50)

cnn_model = CNNModel(num_classes=len(class_names), config=CNN_CONFIG)

# Setup data augmentation
print("🔄 Setting up data augmentation...")
train_generator = data_loader.create_data_generator(
    X_train_cnn, y_train_cnn, 
    batch_size=CNN_CONFIG['training']['batch_size']
)

# Train the model
cnn_model.train(X_train_cnn, y_train_cnn, data_generator=train_generator)

print("\n✅ CNN model training completed!")

In [ ]:
# Plot CNN Training History
if cnn_model.history:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot training & validation accuracy
    ax1.plot(cnn_model.history.history['accuracy'], label='Training Accuracy')
    if 'val_accuracy' in cnn_model.history.history:
        ax1.plot(cnn_model.history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_title('CNN Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True)
    
    # Plot training & validation loss
    ax2.plot(cnn_model.history.history['loss'], label='Training Loss')
    if 'val_loss' in cnn_model.history.history:
        ax2.plot(cnn_model.history.history['val_loss'], label='Validation Loss')
    ax2.set_title('CNN Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    print("📈 CNN training history visualized!")
else:
    print("⚠️ No training history available for CNN")

## 📊 Model Evaluation and Comparison

In [ ]:
# Evaluate Both Models
print("📊 Evaluating both models...")
print("="*50)

# Evaluate SVM
svm_results = svm_model.evaluate(X_test_svm, y_test_svm, class_names)
svm_results['y_true'] = y_test_svm  # Store true labels for confusion matrix

print("\n" + "-"*50)

# Evaluate CNN
cnn_results = cnn_model.evaluate(X_test_cnn, y_test_cnn, class_names)
# Convert categorical labels back to indices for confusion matrix
if len(y_test_cnn.shape) > 1:
    cnn_results['y_true'] = np.argmax(y_test_cnn, axis=1)
else:
    cnn_results['y_true'] = y_test_cnn

print("\n✅ Model evaluation completed!")

In [ ]:
# Detailed Timing Analysis
print("⏱️ Detailed timing analysis...")
print("="*50)

# Sample for inference timing
sample_size = min(TIMING_CONFIG['inference_samples'], len(X_test_svm))
X_sample_svm = X_test_svm[:sample_size]
X_sample_cnn = X_test_cnn[:sample_size]

# Measure inference times
print(f"📊 Measuring inference times on {sample_size} samples...")
svm_timing = svm_model.measure_inference_time(X_sample_svm, TIMING_CONFIG['timing_runs'])
cnn_timing = cnn_model.measure_inference_time(X_sample_cnn, TIMING_CONFIG['timing_runs'])

print(f"\n🔧 SVM Inference Timing (avg of {TIMING_CONFIG['timing_runs']} runs):")
print(f"  Mean: {svm_timing['mean_time']*1000:.3f} ms/sample")
print(f"  Std:  {svm_timing['std_time']*1000:.3f} ms/sample")
print(f"  Min:  {svm_timing['min_time']*1000:.3f} ms/sample")
print(f"  Max:  {svm_timing['max_time']*1000:.3f} ms/sample")

print(f"\n🧠 CNN Inference Timing (avg of {TIMING_CONFIG['timing_runs']} runs):")
print(f"  Mean: {cnn_timing['mean_time']*1000:.3f} ms/sample")
print(f"  Std:  {cnn_timing['std_time']*1000:.3f} ms/sample")
print(f"  Min:  {cnn_timing['min_time']*1000:.3f} ms/sample")
print(f"  Max:  {cnn_timing['max_time']*1000:.3f} ms/sample")

# Update results with detailed timing
svm_results.update(svm_timing)
cnn_results.update(cnn_timing)

print("\n✅ Detailed timing analysis completed!")

## 📈 Results Visualization

In [ ]:
# Model Comparison Visualizations
class ModelEvaluator:
    def __init__(self):
        # Set plotting style
        plt.style.use('default')
        sns.set_palette("husl")
    
    def plot_accuracy_comparison(self, svm_results, cnn_results, class_names):
        """Plot accuracy comparison"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # Overall accuracy comparison
        models = ['SVM + HOG', 'CNN']
        accuracies = [svm_results['accuracy'], cnn_results['accuracy']]
        
        colors = ['#3498db', '#e74c3c']
        bars = ax1.bar(models, accuracies, color=colors, alpha=0.7)
        ax1.set_title('Model Accuracy Comparison', fontsize=14, fontweight='bold')
        ax1.set_ylabel('Accuracy')
        ax1.set_ylim(0, 1)
        
        # Add value labels on bars
        for bar, acc in zip(bars, accuracies):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # Per-class accuracy comparison
        svm_report = svm_results['classification_report']
        cnn_report = cnn_results['classification_report']
        
        x = np.arange(len(class_names))
        width = 0.35
        
        svm_class_acc = [svm_report[class_name]['precision'] for class_name in class_names]
        cnn_class_acc = [cnn_report[class_name]['precision'] for class_name in class_names]
        
        ax2.bar(x - width/2, svm_class_acc, width, label='SVM + HOG', color='#3498db', alpha=0.7)
        ax2.bar(x + width/2, cnn_class_acc, width, label='CNN', color='#e74c3c', alpha=0.7)
        
        ax2.set_title('Per-Class Precision Comparison', fontsize=14, fontweight='bold')
        ax2.set_ylabel('Precision')
        ax2.set_xlabel('Classes')
        ax2.set_xticks(x)
        ax2.set_xticklabels(class_names, rotation=45, ha='right')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def plot_time_comparisons(self, svm_results, cnn_results):
        """Plot training and inference time comparisons"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        models = ['SVM + HOG', 'CNN']
        
        # Training time comparison
        training_times = [svm_results['training_time'], cnn_results['training_time']]
        
        colors = ['#3498db', '#e74c3c']
        bars1 = ax1.bar(models, training_times, color=colors, alpha=0.7)
        
        ax1.set_title('Training Time Comparison', fontsize=14, fontweight='bold')
        ax1.set_ylabel('Training Time (seconds)')
        
        # Add value labels on bars
        for bar, time_val in zip(bars1, training_times):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height + max(training_times) * 0.01,
                    f'{time_val:.2f}s', ha='center', va='bottom', fontweight='bold')
        
        ax1.grid(True, alpha=0.3)
        
        # Inference time comparison
        inference_times = [svm_results['inference_time_per_sample'] * 1000, 
                         cnn_results['inference_time_per_sample'] * 1000]  # Convert to ms
        
        bars2 = ax2.bar(models, inference_times, color=colors, alpha=0.7)
        
        ax2.set_title('Inference Time Comparison', fontsize=14, fontweight='bold')
        ax2.set_ylabel('Inference Time per Sample (ms)')
        
        # Add value labels on bars
        for bar, time_val in zip(bars2, inference_times):
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height + max(inference_times) * 0.01,
                    f'{time_val:.3f}ms', ha='center', va='bottom', fontweight='bold')
        
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def plot_confusion_matrices(self, svm_results, cnn_results, class_names):
        """Plot confusion matrices for both models"""
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # SVM Confusion Matrix
        svm_cm = confusion_matrix(svm_results['y_true'], svm_results['predictions'])
        sns.heatmap(svm_cm, annot=True, fmt='d', cmap='Blues', 
                   xticklabels=class_names, yticklabels=class_names,
                   ax=axes[0])
        axes[0].set_title('SVM + HOG Confusion Matrix', fontsize=14, fontweight='bold')
        axes[0].set_xlabel('Predicted')
        axes[0].set_ylabel('Actual')
        
        # CNN Confusion Matrix
        cnn_cm = confusion_matrix(cnn_results['y_true'], cnn_results['predictions'])
        sns.heatmap(cnn_cm, annot=True, fmt='d', cmap='Reds', 
                   xticklabels=class_names, yticklabels=class_names,
                   ax=axes[1])
        axes[1].set_title('CNN Confusion Matrix', fontsize=14, fontweight='bold')
        axes[1].set_xlabel('Predicted')
        axes[1].set_ylabel('Actual')
        
        plt.tight_layout()
        plt.show()
    
    def plot_detailed_metrics(self, svm_results, cnn_results, class_names):
        """Plot detailed metrics comparison"""
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        svm_report = svm_results['classification_report']
        cnn_report = cnn_results['classification_report']
        
        metrics = ['precision', 'recall', 'f1-score']
        
        for idx, metric in enumerate(metrics):
            row = idx // 2
            col = idx % 2
            
            x = np.arange(len(class_names))
            width = 0.35
            
            svm_values = [svm_report[class_name][metric] for class_name in class_names]
            cnn_values = [cnn_report[class_name][metric] for class_name in class_names]
            
            axes[row, col].bar(x - width/2, svm_values, width, label='SVM + HOG', 
                             color='#3498db', alpha=0.7)
            axes[row, col].bar(x + width/2, cnn_values, width, label='CNN', 
                             color='#e74c3c', alpha=0.7)
            
            axes[row, col].set_title(f'{metric.capitalize()} Comparison', fontsize=12, fontweight='bold')
            axes[row, col].set_ylabel(metric.capitalize())
            axes[row, col].set_xlabel('Classes')
            axes[row, col].set_xticks(x)
            axes[row, col].set_xticklabels(class_names, rotation=45, ha='right')
            axes[row, col].legend()
            axes[row, col].grid(True, alpha=0.3)
        
        # Overall metrics comparison
        overall_metrics = ['macro avg', 'weighted avg']
        
        x = np.arange(len(overall_metrics))
        width = 0.35
        
        svm_vals = [svm_report[om]['precision'] for om in overall_metrics if om in svm_report]
        cnn_vals = [cnn_report[om]['precision'] for om in overall_metrics if om in cnn_report]
        
        if svm_vals and cnn_vals:
            axes[1, 1].bar(x - width/2, svm_vals, width, label='SVM + HOG', 
                          color='#3498db', alpha=0.7)
            axes[1, 1].bar(x + width/2, cnn_vals, width, label='CNN', 
                          color='#e74c3c', alpha=0.7)
        
        axes[1, 1].set_title('Overall Metrics Comparison', fontsize=12, fontweight='bold')
        axes[1, 1].set_ylabel('Precision')
        axes[1, 1].set_xlabel('Metric Type')
        axes[1, 1].set_xticks(x)
        axes[1, 1].set_xticklabels(overall_metrics, rotation=45, ha='right')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Create evaluator and generate visualizations
evaluator = ModelEvaluator()

print("📈 Generating comparison visualizations...")
print("="*50)

# Generate all visualizations
evaluator.plot_accuracy_comparison(svm_results, cnn_results, class_names)
evaluator.plot_time_comparisons(svm_results, cnn_results)
evaluator.plot_confusion_matrices(svm_results, cnn_results, class_names)
evaluator.plot_detailed_metrics(svm_results, cnn_results, class_names)

print("\n✅ All visualizations generated!")

## 📋 Comprehensive Comparison Summary

In [ ]:
# Generate Comprehensive Comparison Summary
def print_comparison_summary(svm_results, cnn_results):
    """Print comprehensive comparison summary"""
    
    # Calculate comparison metrics
    accuracy_diff = cnn_results['accuracy'] - svm_results['accuracy']
    training_time_ratio = cnn_results['training_time'] / svm_results['training_time']
    inference_time_ratio = cnn_results['inference_time_per_sample'] / svm_results['inference_time_per_sample']
    
    print("\n" + "="*80)
    print("🏆 COMPREHENSIVE MODEL COMPARISON SUMMARY")
    print("="*80)
    
    print(f"\n📊 ACCURACY COMPARISON:")
    print(f"  SVM + HOG: {svm_results['accuracy']:.4f}")
    print(f"  CNN:       {cnn_results['accuracy']:.4f}")
    print(f"  Difference: {accuracy_diff:.4f} {'(CNN better)' if accuracy_diff > 0 else '(SVM better)'}")
    
    print(f"\n⏱️  TRAINING TIME COMPARISON:")
    print(f"  SVM + HOG: {svm_results['training_time']:.2f} seconds")
    print(f"  CNN:       {cnn_results['training_time']:.2f} seconds")
    print(f"  Ratio:     {training_time_ratio:.2f}x {'(CNN slower)' if training_time_ratio > 1 else '(CNN faster)'}")
    
    print(f"\n🚀 INFERENCE TIME COMPARISON:")
    print(f"  SVM + HOG: {svm_results['inference_time_per_sample']*1000:.3f} ms/sample")
    print(f"  CNN:       {cnn_results['inference_time_per_sample']*1000:.3f} ms/sample")
    print(f"  Ratio:     {inference_time_ratio:.2f}x {'(CNN slower)' if inference_time_ratio > 1 else '(CNN faster)'}")
    
    print(f"\n🏆 RECOMMENDATIONS:")
    if accuracy_diff > 0:
        print("  • CNN achieves higher accuracy")
    else:
        print("  • SVM + HOG achieves higher accuracy")
    
    if training_time_ratio > 1:
        print("  • SVM + HOG trains faster")
    else:
        print("  • CNN trains faster")
    
    if inference_time_ratio > 1:
        print("  • SVM + HOG has faster inference")
    else:
        print("  • CNN has faster inference")
    
    # Overall recommendation
    print(f"\n💡 OVERALL RECOMMENDATION:")
    if accuracy_diff > 0.05:  # CNN significantly better
        print("  • Use CNN for better accuracy (recommended for this dataset)")
    elif accuracy_diff < -0.05:  # SVM significantly better
        print("  • Use SVM + HOG for better accuracy (recommended for this dataset)")
    else:
        if training_time_ratio > 2:  # SVM much faster to train
            print("  • Use SVM + HOG for faster training with comparable accuracy")
        else:
            print("  • Choice depends on your priority: accuracy vs. speed")
    
    print("\n📚 KEY INSIGHTS:")
    print("  • SVM + HOG: Traditional approach, faster training, good for small datasets")
    print("  • CNN: Deep learning approach, potentially higher accuracy, better for large datasets")
    print("  • Data augmentation can significantly improve CNN performance")
    print("  • HOG features capture edge information effectively")
    print("  • CNN learns features automatically from data")
    
    print("="*80)

# Print comprehensive summary
print_comparison_summary(svm_results, cnn_results)

## 💾 Save Results and Model Information

In [ ]:
# Save Model Information and Results
print("💾 Saving model information and results...")
print("="*50)

# Create results directory
results_dir = 'results'
os.makedirs(results_dir, exist_ok=True)

# Prepare comprehensive model information
model_info = {
    'experiment_info': {
        'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
        'gpu_used': len(tf.config.experimental.list_physical_devices('GPU')) > 0,
        'tensorflow_version': tf.__version__,
        'total_experiment_time': svm_results['training_time'] + cnn_results['training_time']
    },
    'dataset_info': {
        'num_classes': len(class_names),
        'class_names': class_names,
        'total_samples': len(X),
        'train_samples': len(X_train_svm),
        'test_samples': len(X_test_svm),
        'image_size': DATASET_CONFIG['image_size'],
        'test_size_ratio': DATASET_CONFIG['test_size']
    },
    'svm_model': {
        'model_info': svm_model.get_model_info(),
        'results': {
            'accuracy': svm_results['accuracy'],
            'training_time': svm_results['training_time'],
            'inference_time_per_sample': svm_results['inference_time_per_sample'],
            'mean_inference_time': svm_results['mean_time'],
            'std_inference_time': svm_results['std_time']
        }
    },
    'cnn_model': {
        'model_info': cnn_model.get_model_info(),
        'results': {
            'accuracy': cnn_results['accuracy'],
            'training_time': cnn_results['training_time'],
            'inference_time_per_sample': cnn_results['inference_time_per_sample'],
            'mean_inference_time': cnn_results['mean_time'],
            'std_inference_time': cnn_results['std_time']
        }
    },
    'comparison_summary': {
        'accuracy_difference': cnn_results['accuracy'] - svm_results['accuracy'],
        'training_time_ratio': cnn_results['training_time'] / svm_results['training_time'],
        'inference_time_ratio': cnn_results['inference_time_per_sample'] / svm_results['inference_time_per_sample'],
        'winner_accuracy': 'CNN' if cnn_results['accuracy'] > svm_results['accuracy'] else 'SVM',
        'winner_training_speed': 'SVM' if svm_results['training_time'] < cnn_results['training_time'] else 'CNN',
        'winner_inference_speed': 'SVM' if svm_results['inference_time_per_sample'] < cnn_results['inference_time_per_sample'] else 'CNN'
    }
}

# Save model information as JSON
with open(os.path.join(results_dir, 'model_comparison.json'), 'w') as f:
    json.dump(model_info, f, indent=2)

print(f"✅ Model information saved to {results_dir}/model_comparison.json")

# Save classification reports
with open(os.path.join(results_dir, 'svm_classification_report.json'), 'w') as f:
    json.dump(svm_results['classification_report'], f, indent=2)

with open(os.path.join(results_dir, 'cnn_classification_report.json'), 'w') as f:
    json.dump(cnn_results['classification_report'], f, indent=2)

print(f"✅ Classification reports saved")

# Save CNN training history if available
if cnn_model.history:
    with open(os.path.join(results_dir, 'cnn_training_history.json'), 'w') as f:
        json.dump(cnn_model.history.history, f, indent=2)
    print(f"✅ CNN training history saved")

print("\n📁 All results saved successfully!")

## 📥 Download Results and Components

In [ ]:
# Download Functions
from google.colab import files
import pickle

def create_download_packages():
    """Create separate download packages for different components"""
    
    print("📦 Creating download packages...")
    
    # 1. Results Package
    results_files = [
        'results/model_comparison.json',
        'results/svm_classification_report.json',
        'results/cnn_classification_report.json'
    ]
    
    if os.path.exists('results/cnn_training_history.json'):
        results_files.append('results/cnn_training_history.json')
    
    with zipfile.ZipFile('results_package.zip', 'w') as zipf:
        for file in results_files:
            if os.path.exists(file):
                zipf.write(file, os.path.basename(file))
    
    # 2. Models Package
    print("💾 Saving trained models...")
    
    # Save SVM model
    with open('svm_model.pkl', 'wb') as f:
        pickle.dump(svm_model, f)
    
    # Save CNN model
    cnn_model.model.save('cnn_model.h5')
    
    # Create models zip
    with zipfile.ZipFile('models_package.zip', 'w') as zipf:
        zipf.write('svm_model.pkl')
        zipf.write('cnn_model.h5')
    
    # 3. Dataset Package
    print("📊 Packaging dataset...")
    
    with zipfile.ZipFile('dataset_package.zip', 'w') as zipf:
        for root, dirs, files in os.walk('dataset'):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, 'dataset')
                zipf.write(file_path, f'dataset/{arcname}')
    
    # 4. Configuration Package
    config_data = {
        'DATASET_CONFIG': DATASET_CONFIG,
        'SVM_HOG_CONFIG': SVM_HOG_CONFIG,
        'CNN_CONFIG': CNN_CONFIG,
        'EVALUATION_CONFIG': EVALUATION_CONFIG,
        'TIMING_CONFIG': TIMING_CONFIG
    }
    
    with open('configurations.json', 'w') as f:
        json.dump(config_data, f, indent=2)
    
    with zipfile.ZipFile('config_package.zip', 'w') as zipf:
        zipf.write('configurations.json')
    
    print("✅ All download packages created!")
    
    return {
        'results': 'results_package.zip',
        'models': 'models_package.zip',
        'dataset': 'dataset_package.zip',
        'config': 'config_package.zip'
    }

# Create download packages
download_packages = create_download_packages()

print("\n📥 Download packages ready!")
print("Click on the links below to download:")
for package_type, filename in download_packages.items():
    print(f"  • {package_type.capitalize()}: {filename}")

In [ ]:
# Download Results Package
print("📥 Downloading Results Package...")
files.download('results_package.zip')
print("✅ Results package downloaded!")

In [ ]:
# Download Models Package
print("📥 Downloading Models Package...")
files.download('models_package.zip')
print("✅ Models package downloaded!")

In [ ]:
# Download Dataset Package
print("📥 Downloading Dataset Package...")
files.download('dataset_package.zip')
print("✅ Dataset package downloaded!")

In [ ]:
# Download Configuration Package
print("📥 Downloading Configuration Package...")
files.download('config_package.zip')
print("✅ Configuration package downloaded!")

## 📋 Instructions for GitHub Repository Upload

### 🔄 To upload the dataset to your exam repository:

1. **Download the dataset package** using the cell above
2. **Extract the dataset_package.zip** file
3. **Upload the extracted 'dataset' folder** to your GitHub exam repository
4. **Commit and push** the changes

### 📁 Repository Structure Recommendation:
```
your-exam-repo/
├── dataset/
│   ├── normal/
│   │   ├── normal_001.png
│   │   ├── normal_002.png
│   │   └── ...
│   ├── cheating/
│   │   ├── cheating_001.png
│   │   ├── cheating_002.png
│   │   └── ...
│   └── looking_around/
│       ├── looking_around_001.png
│       ├── looking_around_002.png
│       └── ...
├── SVM_vs_CNN_Image_Classification_Comparison.ipynb
├── README.md
└── results/
```

### 🚀 Usage Instructions:

1. **Upload this notebook** to your GitHub repository
2. **Open in Google Colab** by clicking the "Open in Colab" button
3. **Run all cells** to reproduce the complete comparison
4. **Download results** using the download cells above

### 📊 What you get:

- **Complete comparison** between SVM+HOG and CNN approaches
- **Comprehensive visualizations** and performance metrics
- **Trained models** ready for deployment
- **Detailed analysis** and recommendations
- **Reproducible results** with consistent random seeds


## 🎯 Final Summary

### ✅ Experiment Completed Successfully!

This notebook has successfully:

1. **🔍 Created a synthetic dataset** with 3 classes and distinct visual patterns
2. **🏗️ Implemented two different approaches:**
   - SVM with HOG features (traditional computer vision)
   - CNN with data augmentation (deep learning)
3. **📊 Conducted comprehensive comparison** including:
   - Accuracy comparison
   - Training time analysis
   - Inference speed measurements
   - Detailed visualizations
4. **🎮 Utilized GPU acceleration** when available
5. **📥 Provided separate downloads** for all components
6. **📋 Generated detailed reports** and recommendations

### 🏆 Key Insights:

- **SVM + HOG**: Fast training, good for small datasets, interpretable features
- **CNN**: Potentially higher accuracy, automatic feature learning, better scalability
- **GPU acceleration**: Significantly speeds up CNN training
- **Data augmentation**: Improves CNN generalization

### 📚 Next Steps:

1. **Upload dataset** to your GitHub exam repository
2. **Experiment with different configurations** using the config cells
3. **Try with your own dataset** by modifying the data loading section
4. **Deploy models** for real-world applications

---

**🎉 Congratulations! You now have a complete image classification comparison system ready for Google Colab!**
